### Info

Author: Jelle Poland \
Citing: https://doi.org/10.3390/en16145264 \
License: ... \
Github: ...


### Initialisation


In [ ]:
# making things autorelad - needed for Jupyter Kernel
%load_ext autoreload
%autoreload 2
%matplotlib widget

## define the right-path
import sys
import os
# TODO: remove this hardcoding
folder_path = '/home/jellepoland/surfdrive/phd/code/kitesim/'
os.chdir(folder_path)  # This should not be needed
sys.path.append(os.getcwd())



# Time to import the modules
from src.initialisation import load_surfplan, pulley_connectivity
from src.initialisation.input_classes import input_VSM , input_bridle_aero, input_structural_solver
from src.initialisation import input_particleSystem,particles_with_rotational_resistance
from src.particleSystem.ParticleSystem import ParticleSystem
from src.coupling import coupling_aero2struc, coupling_struc2aero
from src.structural import structural_model, structural_mesher
from src.solver import solver_main, solver_utils
from src.post_processing import functions_print, functions_plot, post_processing_utils, post_processing_main
from src.initialisation import actuation_relations
from src.aerodynamic import VSM, breukels_2D, plate_aero, bridle_line_system_aero, tether_aero

# from test import test_main
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import time
import scipy.optimize
import yaml
import importlib
import pytest
import pandas as pd
import dill

# {} gives units. {{}} is normal {} in Latexs
from IPython.display import display, Latex

## All Immutatbles are stored in the dataclass config, i.e. simulation settings and configuration of the kite 
from src.initialisation.yaml_loader import config

## Mutable variables
# Initializing Mutable Variables
points = config.kite.points_ini
# defining vel_app (vector of vel_app_norm)
vel_app = config.vel_wind - config.vel_kite

# TODO: remove this, has been taken into the run_simulation part
# if (
#     config.kite_name == "V3_25"
# ):  # TODO: Should be generated/imported from the surfplan file instead
#     # ACTUATION
#     depower_tape_extension = actuation_relations.up_to_ld(
#         config.u_p, config.depower_tape_extension_percentage
#     )  # [mm] depower-tape extension
#     # bridle_rest_lengths[1] corresponds to the depower tape
#     bridle_rest_lengths[1] += depower_tape_extension

# Should be the same for each kite
(
    connectivity_matrix,
    wing_connectivity,
) = input_particleSystem.define_connectivity_matrix(config)
params_dict = input_particleSystem.define_params(
    config, wing_connectivity, connectivity_matrix
)
initial_conditions = input_particleSystem.define_initial_conditions_kite(
    config)
points_between_dict = particles_with_rotational_resistance.extract_points_between_dict(
    config
)
if config.is_with_initial_plot:
    functions_plot.plot_initial_geometry(config, points_between_dict)

is_with_rotational_resistance = False
if config.kite_name == "V9_60C":
    is_with_rotational_resistance = True

if is_with_rotational_resistance:
    (
        leadingedge_rotational_resistance_dict,
        strut_rotational_resistance_dict,
    ) = particles_with_rotational_resistance.extract_rotational_resistances_dicts(
        points_between_dict, config
    )
    # first do the struts
    k_bend_strut = 1e10
    params_dict = particles_with_rotational_resistance.initialize_bending_spring(
        k_bend_strut,
        initial_conditions,
        params_dict,
        connectivity_matrix,
        strut_rotational_resistance_dict,
    )
    # secondly do the leading-edge
    k_bend_leadingedge = 1e4
    params_dict = particles_with_rotational_resistance.initialize_bending_spring(
        k_bend_leadingedge,
        initial_conditions,
        params_dict,
        connectivity_matrix,
        leadingedge_rotational_resistance_dict,
    )
# Should be the same for each kite
psystem = ParticleSystem(connectivity_matrix, initial_conditions, params_dict)

# Printing initial dimensions
print(f"scaling-factor: {config.geometric_scaling_factor}")
print(f'ref_chord: {config.kite.ref_chord:.2f}m')
print(f'wing_span: {config.kite.span:.2f}m')
print(f'wing height: {config.kite.height:.2f}m')
print(f'wing area: {config.kite.area_surface:.2f}m2')
print(f'projected_area: {config.kite.area_projected:.2f}m')
print(f'At 25m/s --> Reynolds number: {1e-6*(1.225*25*config.kite.ref_chord)/(1.8e-5):.2f}e6')

# Static-aero


In [ ]:
# Struc --> aero
(
    points_wing_segment_corners_aero_orderded,
    index_transformation_struc_to_aero,
) = coupling_struc2aero.extract_wingpanel_corners_aero_orderded(
    points, config.kite.connectivity.plate_point_indices
)
points_old_method = points_wing_segment_corners_aero_orderded
points_new_method = points[index_transformation_struc_to_aero]
print(f"If true its great! {np.allclose(points_old_method, points_new_method)}")
# creating a dict with key value pairs, to transform from aero to struc
index_transformation_aero_to_struc_dict = {}
for i, value in enumerate(index_transformation_struc_to_aero):
    index_transformation_aero_to_struc_dict[value] = i

# NEW COUPLING ALGORITHM post-2024/02
# Struc --> aero
points_wing_segment_corners_aero_orderded = points[index_transformation_struc_to_aero]
# Wing Aerodynamic
(
    force_aero_wing_VSM,
    moment_aero_wing_VSM,
    F_rel,
    ringvec,
    controlpoints,
    wingpanels,
    rings,
    coord_L,
    coord_refined,
) = VSM.calculate_force_aero_wing_VSM(
    points_wing_segment_corners_aero_orderded, vel_app, input_VSM
)
# Aero --> struc
if config.coupling_method == "NN":
    force_aero_wing = coupling_aero2struc.aero2struc_NN(
        config.aero.n_chordwise_aero_nodes,
        wingpanels,
        force_aero_wing_VSM,
        points_wing_segment_corners_aero_orderded,
        index_transformation_aero_to_struc_dict,
        points,
    )
elif config.coupling_method == "MSc_Oriol":
    force_aero_wing = coupling_aero2struc.aero2struc(
        points,
        config.kite.connectivity.wing_ci,
        config.kite.connectivity.wing_cj,
        config.kite.connectivity.plate_point_indices,
        force_aero_wing_VSM,
        moment_aero_wing_VSM,
        ringvec,
        controlpoints,
    )
else:
    raise ValueError("Coupling method not recognized; wrong name or typo")

# Bridle Aerodynamics
if config.is_with_aero_bridle:
    force_aero_bridle = bridle_line_system_aero.calculate_force_aero_bridle_thedens2022(
        points, vel_app, input_bridle_aero
    )
else:
    force_aero_bridle = [0]
force_aero = force_aero_wing + force_aero_bridle

functions_print.print_aero(points, vel_app, force_aero_wing, force_aero_bridle, config)

#######
points_wing = np.copy(points)
evaluation_point = np.array([max(points_wing[:, 0]) / 2, 0, max(points_wing[:, 2])])
evaluation_point = np.zeros(3)
for i, point in enumerate(points_wing):
    points_wing[i] = point - evaluation_point
    if i not in config.kite.connectivity.plate_point_indices:
        points_wing[i] = np.array([0, 0, 0])
        force_aero[i] = np.array([0, 0, 0])

moments_wing = np.cross(points_wing, force_aero)
print(
    f"Moments Full Wing on mid-span, mid-chord point --> Mx: {moments_wing[:,0].sum():.3f} My: {moments_wing[:,1].sum():.3f} Mz: {moments_wing[:,2].sum():.2f}"
)
for i, point in enumerate(points_wing):
    if points_wing[i][1] > 0:  # taking only the right wing
        points_wing[i] = np.array([0, 0, 0])
        force_aero[i] = np.array([0, 0, 0])

moments_half_wing = np.cross(points_wing, force_aero)
print(
    f"Moments Half Right Wing on mid-span, mid-chord point --> Mx: {moments_half_wing[:,0].sum():.3f} My: {moments_half_wing[:,1].sum():.3f} Mz: {moments_half_wing[:,2].sum():.2f}"
)
print(f"norm of Va: {np.linalg.norm(vel_app):.2f}m/s")
#######

# plotting
post_processing_main.plot(
    [
        [wingpanels, controlpoints, rings, coord_L, F_rel],
        config.kite.wing_rest_lengths_initial,
        config.kite.bridle_rest_lengths_initial,
    ],
    points,
    vel_app,
    config,
)

In [ ]:
len(rings)

In [ ]:
vk = 20
r_0 = 50
r_k = 50
# r_ratio_array = []
# for cp in controlpoints.values():
#     y_controlpoint = cp["coordinates"][1]
#     r_ratio_array.append((r_0 + y_controlpoint) / (r_k))

r_ratio_array = [(r_0 + cp["coordinates"][1]) / r_k for cp in controlpoints]
print(f"r_ratio_array: {r_ratio_array}")
vk_distribution = [vk * r_ratio for r_ratio in r_ratio_array]
print(f"vk_distribution: {vk_distribution}")
print(f"vk_distribution: {np.average(vk_distribution)}")

In [ ]:
# print(f"F_rel: {F_rel}")
# print(f"ringvec: {ringvec}")
evaluation_point = 3
print(f"controlpoints: {controlpoints[evaluation_point]['coordinates']}")
# print(f"wingpanels: {wingpanels[evaluation_point]}")
print(f"rings: {rings[evaluation_point]}")
# print(f"coord_L: {coord_L}")
# print(f"coord_refined: {coord_refined}")
print(controlpoints[0])
print(np.linalg.norm(np.array([0.06774378, -0.97854369, 0.19458425])))

# Testing the new coupling-algorithm


In [ ]:
# external force
# Struc --> aero
from sklearn.neighbors import NearestNeighbors


def order_struc_nodes_right_to_left(points_ini, plate_point_indices):
    # defined from LE-to-TE and from right-to-left
    # TODO: implement this from left-to-right
    # TODO: plates should be named segments...
    # We can get these coordinates from plate_point_indices
    # Ordered from left-to-right like: [idx_left_LE, idx_right_LE, idx_right_TE,idx_left_TE]

    plate_point_indices_R_to_L = np.flipud(plate_point_indices)
    coordinates_plates_LE_to_TE_R_to_L = np.empty(
        (0, 3))  # Initialize an empty array
    index_transformation = []

    for segment_indices in plate_point_indices_R_to_L:
        coordinates_plates_LE_to_TE_R_to_L = np.vstack(
            (
                coordinates_plates_LE_to_TE_R_to_L,
                points_ini[segment_indices[1]],
                points_ini[segment_indices[2]],
            )
        )
        # Keep track of the indices transformation
        index_transformation.extend([segment_indices[1], segment_indices[2]])

    coordinates_plates_LE_to_TE_R_to_L = np.vstack(
        (
            coordinates_plates_LE_to_TE_R_to_L,
            points_ini[plate_point_indices[0][0]],
            points_ini[plate_point_indices[0][3]],
        )
    )
    # Add the indices of the last two points
    index_transformation.extend(
        [plate_point_indices[0][0], plate_point_indices[0][3]])

    return coordinates_plates_LE_to_TE_R_to_L, np.array(index_transformation)


# Apply the transformation and get index transformation array
points_left_to_right, index_transformation_struc_to_aero = (
    order_struc_nodes_right_to_left(
        points, config.kite.connectivity.plate_point_indices
    )
)
print(
    f"index_transformation_struc_to_aero: {index_transformation_struc_to_aero}")

points_left_to_right = coupling_struc2aero.order_struc_nodes_right_to_left(
    points, config.kite.connectivity.plate_point_indices
)

print(
    f"reconstructed_points - transformed_points: {points[index_transformation_struc_to_aero]-points_left_to_right}"
)

# creating a dict with key value pairs
index_transformation_aero_to_struc_dict = {}
for i, value in enumerate(index_transformation_struc_to_aero):
    index_transformation_aero_to_struc_dict[value] = i
print(
    f"index_transformation_aero_to_struc_dict: {index_transformation_aero_to_struc_dict}"
)

# Wing Aerodynamic
(
    force_aero_wing_VSM,
    moment_aero_wing_VSM,
    F_rel,
    ringvec,
    controlpoints,
    wingpanels,
    rings,
    coord_L,
    coord_refined,
) = VSM.calculate_force_aero_wing_VSM(points_left_to_right, vel_app, input_VSM)

# TODO: new feature
# Distributing VSM-loads chordwise


def calculate_midpoints(wingpanels):
    midpoints = []
    for panel in wingpanels:
        p1, p2, p3, p4 = panel["p1"], panel["p2"], panel["p3"], panel["p4"]
        # Calculate midpoint of the trailing edge
        point_mid_te = p3 + 0.5 * (p4 - p3)
        # Calculate midpoint of the leading edge
        point_mid_le = p1 + 0.5 * (p2 - p1)
        midpoints.append({"point_mid_te": point_mid_te,
                         "point_mid_le": point_mid_le})
    return midpoints


def interpolate_chordwise_points(midpoints, n_points):
    chordwise_points = np.empty(
        (len(midpoints), n_points, 3)
    )  # Initialize array to hold chordwise points
    for panel_idx, panel in enumerate(midpoints):
        point_mid_le = panel["point_mid_le"]
        point_mid_te = panel["point_mid_te"]

        # Linearly interpolate between point_mid_le and point_mid_te
        for i in range(n_points):
            # Calculate interpolation factor
            t = i / (n_points - 1)
            # Linear interpolation
            interpolated_point = (1 - t) * point_mid_le + t * point_mid_te
            chordwise_points[panel_idx, i] = interpolated_point
    return chordwise_points


def generate_distribution(n_points):
    x = np.linspace(0, 1, n_points)
    y = np.zeros_like(x)
    peak_index = int(len(x) * 0.25)
    y[:peak_index] = x[:peak_index] / (0.25)
    y[peak_index:] = (1 - x[peak_index:]) / (0.75)
    return x, y / np.sum(y)


def distribute_force_aero_chordwise(force_aero_wing_VSM, wingpanels, n_points):

    midpoints = calculate_midpoints(wingpanels)
    chordwise_points = interpolate_chordwise_points(midpoints, n_points)
    chordwise_distribution, percentage_distribution = generate_distribution(
        n_points)

    force_aero_wing_VSM_distributed_chordwise = np.zeros(
        (len(force_aero_wing_VSM), n_points, 3)
    )
    for i, points in enumerate(chordwise_points):
        for j, point in enumerate(points):
            force_aero_wing_VSM_distributed_chordwise[i, j] = (
                force_aero_wing_VSM[i] * percentage_distribution[j]
            )
    return force_aero_wing_VSM_distributed_chordwise


def map_aerodynamic_to_structural(
    aero_nodes,
    struct_nodes,
    aero_forces,
):

    # Step 1: Find nearest structural node for each aerodynamic node

    # Fit nearest neighbor model
    nn_model = NearestNeighbors(
        n_neighbors=1, algorithm="auto").fit(struct_nodes)

    # Find nearest structural node for each aerodynamic node
    distances, corresponding_nodes_indices = nn_model.kneighbors(aero_nodes)

    # flatten indices
    corresponding_nodes_indices = corresponding_nodes_indices.flatten()

    # Step 2: transfer aerodynamic forces to structural nodes

    # Initialize array to hold transferred forces
    transferred_forces = np.zeros_like(struct_nodes)

    # Assign aerodynamic forces to corresponding structural nodes
    for i, aero_force in enumerate(aero_forces):
        struct_node_index = corresponding_nodes_indices[i]
        transferred_forces[struct_node_index] += aero_force

    # Return indices of nearest structural nodes
    return transferred_forces


n_points = 5
force_aero_wing_VSM_distributed_chordwise = distribute_force_aero_chordwise(
    force_aero_wing_VSM, wingpanels, n_points
)

print(f"shape: chordwise_points: {np.shape(chordwise_points)}")
print(f"shape: points_left_to_right: {np.shape(points_left_to_right)}")
print(
    f"shape: force_aero_wing_VSM_distributed_chordwise: {np.shape(force_aero_wing_VSM_distributed_chordwise.reshape(-1, 3))}"
)

force_aero_wing_cornes_aero_mesh = map_aerodynamic_to_structural(
    chordwise_points,
    points_left_to_right,
    force_aero_wing_VSM_distributed_chordwise.reshape(-1, 3),
)

force_aero_wing_corners_struc_mesh = np.zeros_like(points)
for key, value in index_transformation_aero_to_struc_dict.items():
    force_aero_wing_corners_struc_mesh[key] = force_aero_wing_cornes_aero_mesh[value]

In [ ]:
# print(force_aero_wing_cornes_aero_mesh)
print(force_aero_wing_corners_struc_mesh)

In [ ]:
evaluation_point = 1

midpoints = calculate_midpoints(wingpanels)
chordwise_points = interpolate_chordwise_points(midpoints, n_points)

print(f"wingpanels: {wingpanels}")
print(f"len(wingpanels): {len(wingpanels)}")
print(f"len controlpoints: {len(controlpoints)}")
print(f"len rings: {len(rings)}")
print(f"len coord_L: {len(coord_L)}")
print(f"len coord_refined: {len(coord_refined)}")

print(f"len midpoints: {len(midpoints)}")
print(f"len chordwise_points: {len(chordwise_points)}")
print(f"len chordwise_points: {len(chordwise_points[0])}")

print(f"point to be evualated: {evaluation_point}")

print(f"wingpanels[0][p1]: {wingpanels[evaluation_point]['p1']}")
print(f"wingpanels[0][p2]: {wingpanels[evaluation_point]['p2']}")
# print(f"controlpoints[0]: {controlpoints[0]}")
# print(f"rings[0]: {rings[0]}")
# print(f"coord_L[0]: {coord_L[0]}")
# print(f"coord_refined[0]: {coord_refined[0]}")
print(
    f"midpoints[0][point_mid_le]: {midpoints[evaluation_point]['point_mid_le']}")
print(f"wingpanels[0][p3]: {wingpanels[evaluation_point]['p3']}")
print(f"wingpanels[0][p4]: {wingpanels[evaluation_point]['p4']}")
print(
    f"midpoints[0][point_mid_te]: {midpoints[evaluation_point]['point_mid_te']}")
print(f"chordwise_points[0]: {chordwise_points[evaluation_point]}")

print(
    f"np.shape(force_aero_wing_VSM_distributed_chordwise): {np.shape(force_aero_wing_VSM_distributed_chordwise)}"
)
print(f"force_aero_wing_VSM[0]: {force_aero_wing_VSM[evaluation_point]}")
print(
    f"force_aero_wing_VSM_distributed_chordwise[0]: {force_aero_wing_VSM_distributed_chordwise[evaluation_point]}"
)

print(
    f"sum(force_aero_wing_VSM_distributed_chordwise[0] -x: {np.sum(force_aero_wing_VSM_distributed_chordwise[evaluation_point][:,0])}"
)
print(
    f"sum(force_aero_wing_VSM_distributed_chordwise[0] -y: {np.sum(force_aero_wing_VSM_distributed_chordwise[evaluation_point][:,1])}"
)
print(
    f"sum(force_aero_wing_VSM_distributed_chordwise[0] -z: {np.sum(force_aero_wing_VSM_distributed_chordwise[evaluation_point][:,2])}"
)

In [ ]:
force_aero_wing_VSM

In [ ]:
print(f"len(points_left_to_right): {len(points_left_to_right)}")
print(f"len(wingpanels): {len(wingpanels)}")
print(
    f"len(config.kite.connectivity.plate_point_indices): {len(config.kite.connectivity.plate_point_indices)}"
)
print(f"len(force_aero_wing_VSM): {len(force_aero_wing_VSM)}")
print(f"len(points): {len(points)}")

In [ ]:
transformed_points = np.copy(points_left_to_right)
points_ini = np.copy(points)
plate_point_indices = np.copy(config.kite.connectivity.plate_point_indices)

In [ ]:
import numpy as np
from scipy.spatial.distance import cdist

# Assuming transformed_points and points_ini are already defined

# Calculate pairwise distances between points_ini and transformed_points
distances = cdist(points_ini, transformed_points)

# Find the index of the closest point in transformed_points for each point in points_ini
closest_indices = np.argmin(distances, axis=1)

# Create an index transformation array
index_transformation = np.zeros(len(points_ini), dtype=int)
index_transformation[np.arange(len(points_ini))] = closest_indices

# Test the index transformation array
reconstructed_points = transformed_points[index_transformation]

# Verify if the transformation is correct by comparing points_ini with reconstructed_points
print(
    np.allclose(points_ini, reconstructed_points)
)  # This should print True if the transformation is correct

In [ ]:
import numpy as np

# defining the coordinates of the corner points of the structural mesh panels


def order_struc_nodes_right_to_left(points_ini, plate_point_indices):
    # defined from LE-to-TE and from right-to-left
    # TODO: implement this from left-to-right
    # TODO: plates should be named segments...
    # We can get these coordinates from plate_point_indices
    # Ordered from left-to-right like: [idx_left_LE, idx_right_LE, idx_right_TE,idx_left_TE]

    plate_point_indices_R_to_L = np.flipud(plate_point_indices)
    coordinates_plates_LE_to_TE_R_to_L = np.empty(
        (0, 3))  # Initialize an empty array
    index_transformation = []

    for segment_indices in plate_point_indices_R_to_L:
        coordinates_plates_LE_to_TE_R_to_L = np.vstack(
            (
                coordinates_plates_LE_to_TE_R_to_L,
                points_ini[segment_indices[1]],
                points_ini[segment_indices[2]],
            )
        )
        # Keep track of the indices transformation
        index_transformation.extend([segment_indices[1], segment_indices[2]])

    coordinates_plates_LE_to_TE_R_to_L = np.vstack(
        (
            coordinates_plates_LE_to_TE_R_to_L,
            points_ini[plate_point_indices[0][0]],
            points_ini[plate_point_indices[0][3]],
        )
    )
    # Add the indices of the last two points
    index_transformation.extend(
        [plate_point_indices[0][0], plate_point_indices[0][3]])

    return coordinates_plates_LE_to_TE_R_to_L, np.array(index_transformation)


# Apply the transformation and get index transformation array
transformed_points, index_transformation = order_struc_nodes_right_to_left(
    points_ini, plate_point_indices
)

# Test the index transformation array
reconstructed_points = points_ini[index_transformation]
print(
    f"reconstructed_points - transformed_points: {reconstructed_points-transformed_points}"
)
# creating a dict with key value pairs
opposite_index_dict = {}
for i, value in enumerate(index_transformation):
    opposite_index_dict[value] = i
print(f"opposite_index_dict: {opposite_index_dict}")

In [ ]:
print(len(points))
print(len(points_left_to_right))
print(np.shape(points_left_to_right))
# print(points)
print(config.kite.connectivity.plate_point_indices)

plate_points = []
for idx_list in config.kite.connectivity.plate_point_indices:
    for idx in idx_list:
        plate_points.append(points[idx])

for idx, (point, point_left_to_right) in enumerate(
    zip(plate_points, points_left_to_right)
):
    print(f"idx: {idx} point: {point} point_left_to_right: {point_left_to_right}")